In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import glob
import dask
import datetime
from dateutil.relativedelta import relativedelta
from functools import partial
import pandas as pd

dask.config.set(**{'array.slicing.split_large_chunks': True})

### Set up basic info

In [2]:
ystart = 1970 # start year of hindcasts 
yend = 2020 # end year of hindcasts
nyears = yend - ystart + 1
initmon=11
initmonstr = str(initmon).zfill(2)
nmems = 20 # the number of hindcast members
memstr = [str(i).zfill(3) for i in np.arange(1,nmems+1,1)] # generating member strings
#---!!!! ADD 100 for Nov initialization !!! I haven't done this yet
#memstr = [str(i+100).zfill(3) for i in np.arange(1,nmems+1,1)]

topdir = "/glade/campaign/cesm/development/espwg/SMYLE-CW3E/timeseries/monthly/TS/"
expname = "b.e21.BSMYLE-CW3E"
outpath="/glade/campaign/cgd/cas/islas/python/smyle_cw3e/DATA_SORT/output_sst/"

### Set up the dask cluster

In [3]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '10GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=10GB',
    project='P04010022',
    walltime='01:00:00',
    interface='mgt')

# scale up
cluster.scale(20)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [4]:
client

<Client: 'tcp://10.18.206.54:41123' processes=19 threads=19, memory=176.89 GiB>

In [5]:
cluster

PBSCluster(552693e6, 'tcp://10.18.206.54:41123', workers=20, threads=20, memory=186.20 GiB)

### Set up the filelist

In [6]:
files = []
for iyear in np.arange(ystart,yend+1,1):
    filest = [glob.glob(topdir+expname+"*."+imem+".cam.h0.TS."+str(iyear)+initmonstr+'-*.nc')[0] for imem in memstr ]
    files.append(filest)

In [7]:
def preprocessor(ds):
    # sort out the times so that each member has the time 
    timebndavg = np.array(ds.time_bnds, dtype='datetime64[s]').view('i8').mean(axis=1).astype('datetime64[s]')
    ds['time'] = timebndavg
    datestart = pd.to_datetime("1970-"+str(ds.isel(time=0).time.dt.month.values).zfill(2)+"-"+str(ds.isel(time=0).time.dt.day.values).zfill(2),
                               format="%Y-%m-%d")
    time = [ datestart + relativedelta(months = int(i)) for i in np.arange(0,ds.time.size,1) ] 
    ds['time'] = time
    return ds

In [8]:
dat = xr.open_mfdataset(files, combine='nested', concat_dim=['init_year', 'M'],
                        parallel=True, data_vars=['TS'], coords='minimal', compat='override', preprocess=partial(preprocessor))
dat['init_year'] = np.arange(ystart,yend+1,1)
dat = dat.TS
dat.load().to_netcdf(outpath+'SST_BSMYLE-CW3E-L32_mon_init'+initmonstr+'.nc')

In [9]:
cluster.close()